# Inventories

Inventories are a fundamental doc entrypoint for our infrastructures. 

They contain a lot of informations, including:
    
    - ansible_user
    - configuration variables
    - host grouping eg. by geographical zones
    
    

In [1]:
cd exercise-04

/notebooks/exercise-04


In [5]:
!cat inventory

# I can group hosts in inventory

[web:children]
web_rome
web_milan

[web_rome:children]
web_rome_test
web_rome_prod

# further host variables
[web_rome:vars]
ansible_username=root 

# Connect with docker 
[web_rome_test:vars]
ansible_connection=docker
ansible_docker_extra_args: " -H=tcp://172.17.0.1:2375 "



# The actual host reference
[web_rome_test]
ansible101_web_1


[web_rome_prod]
172.23.0.[3:4]

[web_milan]
172.24.0.[5:6]


In [10]:
!ansible -m ping -i inventory all


ansible101_web_1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.24.0.6 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.24.0.6 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.23.0.3 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.3 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.24.0.5 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.24.0.5 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.23.0.4 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.4 port 22: Connection timed out\r\n", 
    "unreachable": true
}


In [22]:
#To create custom inventory scripts just use python ;) and set it in

!grep inventory  ansible.cfg #inventory = ./docker-inventory.py


inventory = inventory-docker.py


In [26]:
# List our containers. Note: this only works with docker-compose containers.
from __future__ import print_function
import docker
c=docker.Client(base_url="http://172.17.0.1:2375")
container_fmt = lambda x: (
    x['Names'][0][1:],
    x['Labels']['com.docker.compose.service'], 
    x['NetworkSettings']['Networks']['bridge']['IPAddress'],
)

for x in c.containers():
    print(*container_fmt(x), sep='\t\t')

ansible101_web_6		web		172.17.0.9
ansible101_web_3		web		172.17.0.8
ansible101_web_4		web		172.17.0.7
ansible101_web_5		web		172.17.0.6
ansible101_ansible_1		ansible		172.17.0.5
ansible101_web_2		web		172.17.0.4
ansible101_web_1		web		172.17.0.3
ansible101_dev_1		dev		172.17.0.2


In [34]:
# Ansible accepts
import json

inventories = {
    'web': {
        'hosts': ['ws-1', 'ws-2'],
    },
    'db': {
        'hosts': ['db-1', 'db-2'],
    }
}

# like this 
print(json.dumps(inventories, indent=1))
      

{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ]
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


In [35]:
# You can pass variables to generated inventories too
inventories['web']['host_vars'] = {
    'ansible_ssh_common_args': ' -o GSSApiAuthentication=no'
}

print(json.dumps(inventories, indent=1))


{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ], 
  "host_vars": {
   "ansible_ssh_common_args": " -o GSSApiAuthentication=no"
  }
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


In [40]:
# Exercise: using the above code, create a custom inventory-docker.py that
# - connects via docker to web hosts
# - connects via 



In [1]:
# Test our custom inventory-docker.py file
!ansible -m ping -i inventory-docker.py all --limit ansible


 [WARNING]: Host file not found: inventory-docker.py

 [WARNING]: provided hosts list is empty, only localhost is available

 [WARNING]: No hosts matched, nothing to do



In [ ]:
# Now modify the inventory-docker.py to skip StrictHostKeyChecking only on web hosts.